In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd ../content/drive/MyDrive/Wanted/SUM

/content/drive/MyDrive/Wanted/SUM


# Import Libraries

In [ ]:
import pandas as pd
import json
from tqdm.notebook import tqdm
import re

# Extract Sports Articles from AI HUB Data

- AI HUB 문서추출 데이터셋으로부터 카테고리가 '스포츠 기사'에 해당하는 데이터만 추출

In [ ]:
def extract_sports_articles(in_file_path):

    with open(f'{in_file_path}') as json_in:
        json_data = json.load(json_in)

    sports_articles = []

    for article in tqdm(json_data['documents']):
        if article['category'] == '스포츠':
            sports_articles.append(article)

    return sports_articles

In [ ]:
train_sports = extract_sports_articles('data/train_original.json') # AI HUB train dataset
valid_sports = extract_sports_articles('data/valid_original.json') # AI HUB valid dataset

  0%|          | 0/243983 [00:00<?, ?it/s]

  0%|          | 0/30122 [00:00<?, ?it/s]

In [ ]:
len(train_sports), len(valid_sports)

(5174, 392)

In [ ]:
# Save AI HUB Sports News Data

train_sports_dict, valid_sports_dict = dict(), dict()
train_sports_dict['documents'] = train_sports
valid_sports_dict['documents'] = valid_sports

with open('data/train_sports.json', 'w', encoding='utf-8') as json_out:
    json.dump(train_sports_dict, json_out, ensure_ascii=False, indent='\t')

with open('data/valid_sports.json', 'w', encoding='utf-8') as json_out:
    json.dump(valid_sports_dict, json_out, ensure_ascii=False, indent='\t')

## Create Data Input for Summarization Models

In [ ]:
# Load AI HUB Sports News Data

with open('data/train_sports.json') as json_out:
    train = json.load(json_out)

with open('data/valid_sports.json') as json_out:
    valid = json.load(json_out)

### BERTSUM INPUT

In [ ]:
def create_input_for_bertsum(in_file_path):

    # Load Data
    with open(f'{in_file_path}') as json_in:
        json_data = json.load(json_in)

    data_points = []

    for article in tqdm(json_data['documents']):
        document = []
        onehot_idx = []
        summary_idx = []
        sents = []
        extractive_idx = article['extractive']
        for item in article['text']:
            sents.extend(item)

        # Create Document
        for sent in sents:
            if sent:
                sent_content = sent['sentence']
                document.append(sent_content)

            else:
                continue

        document = '\t'.join(document)

        # Create Data Point (document, summary, summary_idx, onehot_idx)
        summary = []
        true_sent_idx = -1

        for sent in sents:
            if sent:
                sent_idx = sent['index']
                sent_content = sent['sentence']
                true_sent_idx += 1

                if sent_idx in extractive_idx:
                    label = 1
                    summary_idx.append(true_sent_idx)
                    summary.append(sent_content)
                else:
                    label = 0

                onehot_idx.append(label)
            else:
                continue

        summary = '\t'.join(summary)
        data_points.append([document, summary, summary_idx, onehot_idx])


    # DataFrame으로 변환
    df = pd.DataFrame(data_points,
                      columns=['CONTENT', 'LABEL', 'SUMMARY_IDX', 'SUMMARY_ONEHOT_IDX'])

    return df

In [ ]:
train_df = create_input_for_bertsum('data/train_sports.json')
valid_df = create_input_for_bertsum('data/valid_sports.json')

  0%|          | 0/5174 [00:00<?, ?it/s]

  0%|          | 0/392 [00:00<?, ?it/s]

In [ ]:
# Train Data Add Columns
train_df['CONTENT_LEN'] = train_df['CONTENT'].apply(lambda x: len(x.split('\t')))
train_df['ONEHOT_LEN'] = train_df['SUMMARY_ONEHOT_IDX'].apply(lambda x: len(x))
train_df['SUMMARY_CNT'] = train_df['SUMMARY_IDX'].apply(lambda x: len(x))

# Valid Data Add Columns
valid_df['CONTENT_LEN'] = valid_df['CONTENT'].apply(lambda x: len(x.split('\t')))
valid_df['ONEHOT_LEN'] = valid_df['SUMMARY_ONEHOT_IDX'].apply(lambda x: len(x))
valid_df['SUMMARY_CNT'] = valid_df['SUMMARY_IDX'].apply(lambda x: len(x))

In [ ]:
# Exclude Outliers

train_outlier_idx, valid_outlier_idx = [], []

# Exclude Train Outliers
train_outlier_idx.extend(train_df[train_df['CONTENT_LEN'] != train_df['ONEHOT_LEN']].index) # 문서의 길이와 one-hot encoding 길이가 다른 데이터 제외
train_outlier_idx.extend(train_df[train_df['SUMMARY_CNT'] != 3].index) # summary의 문장 개수가 3개가 아닌 데이터 제외

# Exclude Valid Outliers
valid_outlier_idx.extend(valid_df[valid_df['CONTENT_LEN'] != valid_df['ONEHOT_LEN']].index)
valid_outlier_idx.extend(valid_df[valid_df['SUMMARY_CNT'] != 3].index)

train_outlier_idx, valid_outlier_idx

([1019, 4371], [])

In [ ]:
train_df = train_df.drop(index=train_outlier_idx).reset_index(drop=True)[['CONTENT', 'LABEL', 'SUMMARY_IDX', 'SUMMARY_ONEHOT_IDX']]
valid_df = valid_df[['CONTENT', 'LABEL', 'SUMMARY_IDX', 'SUMMARY_ONEHOT_IDX']]

len(train_df), len(valid_df)

(5172, 392)

In [ ]:
# Combine Train and Valid Data

total_df = pd.concat([train_df, valid_df])
total_df.head()

,CONTENT,LABEL,SUMMARY_IDX,SUMMARY_ONEHOT_IDX
0,전남드래곤즈 해맞이 다짐…선수 영입 활발\t이성훈 sinawi@hanmail.net...,임직원과 선수단 모두는 이날 구봉산 정상에 올라 일출을 보며 2018년 구단 목표를...,"[3, 5, 7]","[0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
1,"최복음?신승현?\t유지훈?정현수…아시안게임, 세계대회 출전\t이성훈 sinawi@h...",광양시청 볼링팀 소속 최복음·신승현·유지훈·정현수가 태극마트를 달았다.\t광양시청 ...,"[3, 4, 11]","[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0]"
2,한양공고에 2-0 완승…백운기 첫 우승컵 안아\t이성훈 sinawi@hanmail....,제20회 백운기전국고등학교 축구대회 우승은 서울 중경고등학교(감독 최운범)에게 돌아...,"[2, 3, 6]","[0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,지역신문-전남드래곤즈‘상호협력’\t이성훈 sinawi@hanmail.net\t전남드...,전남드래곤즈(사장 신승재)는 지난 6일 상생의 지역문화 구축을 위해 지역 4개 신문...,"[2, 3, 4]","[0, 0, 1, 1, 1, 0, 0, 0, 0]"
4,"배구 7연패, 볼링 7년 만에 우승…종목별로 골고루 성적\t이성훈 sinawi@ha...",광양시는 지난 19일부터 22일까지 영암군 일원에서 열린‘제57회 전라남도 체육대회...,"[2, 3, 4]","[0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
# Save Data

total_df.to_csv('aihub_total_5564_bertsum.csv', index=False)

### MATCHSUM INPUT

In [ ]:
def create_input_for_matchsum(in_file_path):

    # Load Data
    with open(f'{in_file_path}') as json_in:
        json_data = json.load(json_in)

    min_sent_len = 10
    data_points = []

    for article in tqdm(json_data['documents']):
        document = []
        sents = []
        extractive_idx = article['extractive']
        for item in article['text']:
            sents.extend(item)

        # Create Document
        for sent in sents:
            if sent:
                sent_content = sent['sentence']
                document.append(sent_content)

            else:
                continue

        document = ' '.join(document)

        # Create Data Point (sentence, document, label)
        for sent in sents:
            if sent:
                sent_idx = sent['index']
                sent_content = sent['sentence']

                if sent_idx in extractive_idx:
                    label = 1
                else:
                    label = 0

                data_points.append((sent_content, document, label))


    # DataFrame으로 변환
    df = pd.DataFrame(data_points,
                      columns=['sents', 'docs', 'y'])

    return df

In [ ]:
train_df = create_input_for_matchsum('data/train_sports.json')
valid_df = create_input_for_matchsum('data/valid_sports.json')

display(train_df.head())
display(valid_df.head())

  0%|          | 0/5174 [00:00<?, ?it/s]

  0%|          | 0/392 [00:00<?, ?it/s]

,sents,docs,y
0,전남드래곤즈사장 신승재는 지난 일 구봉산 해맞이 행사를 통해 새해 각오를 다졌다.,전남드래곤즈사장 신승재는 지난 일 구봉산 해맞이 행사를 통해 새해 각오를 다졌다. ...,0
1,"신승재 사장은유상철 감독을 비롯한 코칭스텝, 선수단 구성이 마무리 된 만큼 구성원 ...",전남드래곤즈사장 신승재는 지난 일 구봉산 해맞이 행사를 통해 새해 각오를 다졌다. ...,1
2,유상철 감독은구봉산의 정기를 받아 년을 전남드래곤즈의 해로 만들겠다며 각오를 다졌다.,전남드래곤즈사장 신승재는 지난 일 구봉산 해맞이 행사를 통해 새해 각오를 다졌다. ...,0
3,한편 전남은 선수들도 추가 영입했다.,전남드래곤즈사장 신승재는 지난 일 구봉산 해맞이 행사를 통해 새해 각오를 다졌다. ...,1
4,공격력 강화를 위해 페체신 대체자로 중국 리그에서 활약한 하태균도 영입했다.,전남드래곤즈사장 신승재는 지난 일 구봉산 해맞이 행사를 통해 새해 각오를 다졌다. ...,0


,sents,docs,y
0,홀에 깃대를 꽂은 채 퍼팅하는 '깃대 퍼팅'이 주목받고 있다.,홀에 깃대를 꽂은 채 퍼팅하는 '깃대 퍼팅'이 주목받고 있다. 일한국시간 미국...,1
1,일한국시간 미국여자프로골프LPGA투어 시즌 첫 메이저대회인 ANA인스퍼레이션에서 '...,홀에 깃대를 꽂은 채 퍼팅하는 '깃대 퍼팅'이 주목받고 있다. 일한국시간 미국...,0
2,"고진영은 짧든, 길든 거의 대다수 퍼팅을 깃대를 꽂은 채 한다.",홀에 깃대를 꽂은 채 퍼팅하는 '깃대 퍼팅'이 주목받고 있다. 일한국시간 미국...,0
3,고진영은 깃대 퍼팅 애용자로 남녀 프로 투어에서 처음으로 우승한 데 이어 메이저 우...,홀에 깃대를 꽂은 채 퍼팅하는 '깃대 퍼팅'이 주목받고 있다. 일한국시간 미국...,0
4,깃대 퍼팅이 퍼팅 성공률을 높여준다는 공식 통계는 아직 없다.,홀에 깃대를 꽂은 채 퍼팅하는 '깃대 퍼팅'이 주목받고 있다. 일한국시간 미국...,1


#### Create Balanced Dataset (for MATCHSUM)

In [ ]:
# Check Label Counts for Train Data
pos_train_df = train_df[train_df.y == 1]
neg_train_df = train_df[train_df.y == 0]

# Check Label Counts for Valid Data
pos_valid_df = valid_df[valid_df.y == 1]
neg_valid_df = valid_df[valid_df.y == 0]

print(len(pos_train_df), len(neg_train_df))
print(len(pos_valid_df), len(neg_valid_df))

11531 33970
845 2084


In [ ]:
neg_multiplier = 2 # pos의 몇 배가 되는 neg data를 사용할 것인지

neg_train_df_new = neg_train_df.sample(len(pos_train_df)*neg_multiplier)
neg_valid_df_new = neg_valid_df.sample(len(pos_valid_df)*neg_multiplier)
balanced_train_df = pos_train_df.append(neg_train_df_new)
balanced_valid_df = pos_valid_df.append(neg_valid_df_new)

# Save Balanced Datasets
balanced_train_df.to_json('data/train_sports_bdf_ko.json')
balanced_valid_df.to_json('data/valid_sports_bdf_ko.json')